# 还是熟悉的数据

In [1]:
# Pandas is used for data manipulation
import pandas as pd

# Read in data as a dataframe
features = pd.read_csv('data/temps_extended.csv')

# 还是相同的处理

In [2]:
# One Hot Encoding
features = pd.get_dummies(features)

# Extract features and labels
labels = features['actual']
features = features.drop('actual', axis = 1)

# List of features for later use
feature_list = list(features.columns)

# Convert to numpy arrays
import numpy as np

features = np.array(features)
labels = np.array(labels)

# Training and Testing Sets
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, 
                                                                            test_size = 0.25, random_state = 42)

In [3]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1643, 17)
Training Labels Shape: (1643,)
Testing Features Shape: (548, 17)
Testing Labels Shape: (548,)


In [4]:
print('{:0.1f} years of data in the training set'.format(train_features.shape[0] / 365.))
print('{:0.1f} years of data in the test set'.format(test_features.shape[0] / 365.))

4.5 years of data in the training set
1.5 years of data in the test set


## 还是选择那6个重要性比较高的特征


In [5]:
# Names of five importances accounting for 95% of total importance
important_feature_names = ['temp_1', 'average', 'ws_1', 'temp_2', 'friend', 'year']

# Find the columns of the most important features
important_indices = [feature_list.index(feature) for feature in important_feature_names]

# Create training and testing sets with only the important features
important_train_features = train_features[:, important_indices]
important_test_features = test_features[:, important_indices]

# Sanity check on operations
print('Important train features shape:', important_train_features.shape)
print('Important test features shape:', important_test_features.shape)

Important train features shape: (1643, 6)
Important test features shape: (548, 6)


In [6]:
# Use only the most important features
train_features = important_train_features[:]
test_features = important_test_features[:]

# Update feature list for visualizations
feature_list = important_feature_names[:]

### 还是。。。不对，开始调节新的参数

In [7]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 42)

from pprint import pprint

# 打印所有参数
pprint(rf.get_params())

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


# 开始尝试各种参数吧

调参路漫漫，参数的可能组合结果实在太多了，我们还得有章可循，首先登场的是：RandomizedSearchCV()，这个函数可以帮助我们在候选集组合中，不断的随机选择一组合适的参数来建模，并且求其交叉验证后的评估结果。为什么要不断随机的选择呢？按顺序一个个来不是更靠谱嘛，假设咱们有5个参数待定，每个参数都有10种候选值，那一共有多少种可能性呢？这个数字就很大了吧，由于建立模型所花的时间并不少，当数据量大的时候，几小时能完成一次建模就已经不错了，所以我们很难遍历到所有的可能，随机变成了一种策略，让我们大致能得到比较合适的参数组合，该函数所需的所有参数解释都在API文档中有详细说明.

In [27]:
from sklearn.model_selection import RandomizedSearchCV

# 建立树的个数
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# 最大特征的选择方式
max_features = ['auto', 'sqrt']
# 树的最大深度
max_depth = [int(x) for x in np.linspace(10, 20, num = 2)]
max_depth.append(None)
# 节点最小分裂所需样本个数
min_samples_split = [2, 5, 10]
# 叶子节点最小样本数，任何分裂不能让其子节点样本数少于此值
min_samples_leaf = [1, 2, 4]
# 样本采样方法
bootstrap = [True, False]

# Random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

在这个任务中，我们只给大家举例来进行说明，考虑到时间问题，所选的参数的候选值并没有给出太多。这里值得注意的是每一个候选参数的参数空间需要我们好好把控，因为如果这个取值范围给定的不恰当，最好的结果肯定也不会太好，这里可以参考一些经验值或者不断通过实验结果来改变参数空间，这是一个反复的过程，并不是说我们机器学习建模任务就是从前往后的进行，有了实验结果之后，都需要再回过头来反复来对比不同参数，不同预处理方案的。

In [28]:
# 随机选择最合适的参数组合
rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1)

# 执行寻找操作
rf_random.fit(train_features, train_labels)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   39.4s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.4min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='neg_mean_absolute_error',
          verbose=2)

这里给大家解释一下RandomizedSearchCV中常用的参数，其实在API文档中都给出了说明，还是建议大家养成这个查阅文档的习惯。

- Estimator：RandomizedSearchCV这个方法是一个通用的，并不是专为随机森林设计的，所以我们需要指定选择的算法模型是什么。
- Distributions：参数的候选空间，我们之间已经用字典格式给出了所需的参数分布。
- n_iter：随机寻找参数组合的个数，比如在这里我们赋值了100代表接下来要随机找100组参数的组合，在其中找到最好的一个。
- Scoring：评估方法，按照该方法去找到最好的参数组合
- Cv：交叉验证，咱们之前已经唠过了。
- Verbose：打印信息的数量，看自己的需求了。
- random_state：随机种子，为了使得咱们的结果能够一致，排除掉随机成分的干扰，一般我们都会指定成一个值，用你自己的幸运数字就好。
- n_jobs：多线程来跑这个程序，如果是-1就会用所有的，但是可能会有点卡。

即便我把n_jobs设置成了-1，程序运行的还是很慢，因为我们建立100次模型来选择参数，并且还是带有3折交叉验证的，那就相当于300个任务了，结果如下图所示：

In [29]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 1400}

## 评估函数

接下来就对比一下，经过调参后的结果和用默认参数结果的差异，所有默认参数在API中都有说明，比如n_estimators : integer, optional (default=10)，这里就说明在随机森林模型中，默认要建立树的个数是10个。先给出评估标准：

In [30]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape

    print('平均气温误差.',np.mean(errors))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

#### 看看效果吧

老模型

In [31]:
base_model = RandomForestRegressor( random_state = 42)
base_model.fit(train_features, train_labels)
evaluate(base_model, test_features, test_labels)

平均气温误差. 3.91989051095
Accuracy = 93.36%.


#### 新配方（最好的参数）

In [32]:
best_random = rf_random.best_estimator_
evaluate(best_random, test_features, test_labels)

平均气温误差. 3.71143985958
Accuracy = 93.74%.


# Grid Search ，之前不是找到差不多的方案了嘛，再来微调！

可以看到模型的效果提升了一些，但是这已经是上限了嘛？还有没有可以进步的空间了呢？接下来我们又要介绍下位参选选手了：GridSearchCV()，它的意思是进行网络搜索，说白了就是一个一个的遍历，就像我们之前说的组合有多少种，就全部走一遍，其所需的参数都是类似的，没记住的话赶紧先翻一遍API文档：

In [34]:
from sklearn.model_selection import GridSearchCV

# 网络搜索
param_grid = {
    'bootstrap': [True],
    'max_depth': [8,10,12],
    'max_features': ['auto'],
    'min_samples_leaf': [2,3, 4, 5,6],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [800, 900, 1000, 1200]
}

# 选择基本算法模型
rf = RandomForestRegressor()

# 网络搜索
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                           scoring = 'neg_mean_absolute_error', cv = 3, 
                           n_jobs = -1, verbose = 2)

In [35]:
# 执行搜索
grid_search.fit(train_features, train_labels)

Fitting 3 folds for each of 180 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   31.4s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  2.1min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [True], 'max_depth': [8, 10, 12], 'max_features': ['auto'], 'min_samples_leaf': [2, 3, 4, 5, 6], 'min_samples_split': [3, 5, 7], 'n_estimators': [800, 900, 1000, 1200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=2)

In [36]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 12,
 'max_features': 'auto',
 'min_samples_leaf': 6,
 'min_samples_split': 7,
 'n_estimators': 900}

In [37]:
best_grid = grid_search.best_estimator_
evaluate(best_grid, test_features, test_labels)

平均气温误差. 3.6838716214
Accuracy = 93.78%.


## 另一组参赛选手 Grid Search

经过了再调整之后我们的算法模型效果又有了一点提升，虽然只是一小点，但是把每一小步累计在一次就是一个大成绩了。再用网络搜索的时候，遍历的次数太多，我们通常并不把所有的可能性都放进去，而是分成不同的组来分别执行，下面我们再来看看另外一组网络搜索的参赛选手:

In [38]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [12, 15, None],
    'max_features': [3, 4,'auto'],
    'min_samples_leaf': [5, 6, 7],
    'min_samples_split': [7,10,13],
    'n_estimators': [900, 1000, 1200]
}

# 选择算法模型
rf = RandomForestRegressor()

# 继续寻找
grid_search_ad = GridSearchCV(estimator = rf, param_grid = param_grid, 
                           scoring = 'neg_mean_absolute_error', cv = 3, 
                           n_jobs = -1, verbose = 2)

grid_search_ad.fit(train_features, train_labels)

Fitting 3 folds for each of 243 candidates, totalling 729 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 729 out of 729 | elapsed:  2.5min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [True], 'max_depth': [12, 15, None], 'max_features': [3, 4, 'auto'], 'min_samples_leaf': [5, 6, 7], 'min_samples_split': [7, 10, 13], 'n_estimators': [900, 1000, 1200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=2)

In [39]:
grid_search_ad.best_params_

{'bootstrap': True,
 'max_depth': 15,
 'max_features': 4,
 'min_samples_leaf': 6,
 'min_samples_split': 10,
 'n_estimators': 1000}

In [40]:
best_grid_ad = grid_search_ad.best_estimator_
evaluate(best_grid_ad, test_features, test_labels)

平均气温误差. 3.66263669806
Accuracy = 93.82%.


看起来第二组选手要比第一组强一些，经过了这一番折腾之后我们可以把最终选定的所有参数都列出来了，93.82%相当于我们到此最优的一个结果了

## 最终模型

In [42]:
print('最终模型参数:\n')
pprint(best_grid_ad.get_params())

最终模型参数:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': 15,
 'max_features': 4,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 6,
 'min_samples_split': 10,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


平均气温误差. 3.66263669806
Accuracy = 93.82%.


来总结一下我们的调参任务吧：

- 1.参数空间是非常重要的，它会对结果产生决定性影响，所以在开始任务之前，得选择大致一个合适区间，可以参考一些相同任务论文的经验值。
- 2.随机搜索可以更节约时间，尤其是在任务开始阶段，我们并不知道哪一个参数在哪一个位置效果能更好，这样我们可以把参数间隔设置的更大一些，先用随机搜索确定一些大致位置。
- 3.网络搜索相当于地毯式搜索了，当我们得到了大致位置之后，想在这里寻找到最优参数的时候就派上用场了，可以把随机和网络搜索当做一套组合拳，搭配使用。
- 4.最后调参的方法其实还有很多的，比如贝叶斯优化，这个还是蛮有意思的，跟大家简单说一下，想一想我们之前的调参方式，是不是每一个都是独立的进行不会对之后的结果产生任何影响，贝叶斯优化的基本思想在于每一个优化都是在不断积累经验，这样我会慢慢得到最终的解应当在的位置，相当于前一步结果会对后面产生影响了，如果大家对贝叶斯优化感兴趣，可以参考下Hyperopt工具包，用起来也很简便：